In [15]:
import pandas as pd 
LAYER=32

In [16]:
languages = ['cpp','cs','java','js','php','python','sh','ts']

In [25]:
def get_label_df(lang):
    source_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_{LAYER}.parquet'
    label_lang_path = f'data/ds67{lang}/output2/LFCLF_embedding_human_eval_deepseek-ai_deepseek-coder-6.7b-instruct_1_label.parquet'
    # print(source_lang_path)
    # print(label_lang_path)
    df_source = pd.read_parquet(source_lang_path)
    df_label = pd.read_parquet(label_lang_path)
    # print(df_label.head(1))
    result = list()
    for idx,row in df_source.iterrows():
        row['label'] = 1 if df_label.at[idx, 'label'] else 0
        result.append(row)
    return pd.DataFrame(result)    

In [ ]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os

for language in languages:
    test_df = get_label_df(language)
    trains = list()
    for otherlang in languages:
        if otherlang == language:
            continue
        trains.append(get_label_df(otherlang))
    train_df = pd.concat(trains)
    train_df = train_df.sample(frac = 1)
    out_dir_lang = os.path.join(out_dir,language)
    if not os.path.exists(out_dir_lang):
        os.mkdir(out_dir_lang)
    test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
    train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
    print(language,test_df.shape,train_df.shape)
    print(test_df['label'].value_counts())
    print(train_df['label'].value_counts())


In [ ]:
train_df.head(1)

In [ ]:
SETTING = 'cross_language'
out_dir = 'data/data_for_classify'
import os
import re
train = list()
test = list()
dfs = list()
for language in languages:
    dfs.append(get_label_df(language))
data_df = pd.concat(dfs)

with open('data/train.txt') as f:
    train_id = [int(l.strip()) for l in f.readlines()]
with open('data/test.txt') as f:
    test_id = [int(l.strip()) for l in f.readlines()]

for _,row in data_df.iterrows():
    task_id = row['task_id']
    numbers = re.findall(r'\d+', task_id)
    if len(numbers) <= 0:
        train.append(row)
        continue
    if int(numbers[0]) in train_id:
        train.append(row)
    else:
        test.append(row)

train_df = pd.DataFrame(train)
test_df = pd.DataFrame(test)

train_df = train_df.sample(frac = 1)
out_dir_lang = os.path.join(out_dir,'cross_task')
if not os.path.exists(out_dir_lang):
    os.mkdir(out_dir_lang)
test_df.to_parquet(os.path.join(out_dir_lang,'test.parquet'))
train_df.to_parquet(os.path.join(out_dir_lang,'train.parquet'))
print(language,test_df.shape,train_df.shape)
print(test_df['label'].value_counts())
print(train_df['label'].value_counts())


In [ ]:
train_df.head(10)